<a href="https://colab.research.google.com/github/DiegoPincheiraIb/EL4106_Assignments/blob/main/Assignment5/Assignment5_EL4106.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuraciones preliminares

## Importación de librerias:

In [ ]:
from time import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import metrics
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

np.random.seed(42)

## Importación de archivos:

In [ ]:
#Comentar esto si no se usa colaboratory
from google.colab import files
uploaded = files.upload()

Saving Data_Cortex_Nuclear.xls to Data_Cortex_Nuclear.xls


# Procesamiento de datos

In [ ]:
excel_file = 'Data_Cortex_Nuclear.xls'
Datos_Iniciales = pd.read_excel(excel_file)
## ---- Procesamiento de Dataframe original
del Datos_Iniciales['MouseID'], Datos_Iniciales['Behavior'], Datos_Iniciales['Genotype'], Datos_Iniciales['Treatment'], Datos_Iniciales['BCL2_N']
# Se eliminan las filas con datos NaN
Datos_Iniciales_Copia = Datos_Iniciales.copy()
Datos_Iniciales = Datos_Iniciales_Copia.dropna()
print(Datos_Iniciales)

      DYRK1A_N   ITSN1_N    BDNF_N  ...  H3MeK4_N    CaNA_N   class
0     0.503644  0.747193  0.430175  ...  0.128186  1.675652  c-CS-m
1     0.514617  0.689064  0.411770  ...  0.131119  1.743610  c-CS-m
2     0.509183  0.730247  0.418309  ...  0.127431  1.926427  c-CS-m
3     0.442107  0.617076  0.358626  ...  0.146901  1.700563  c-CS-m
4     0.434940  0.617430  0.358802  ...  0.148380  1.839730  c-CS-m
...        ...       ...       ...  ...       ...       ...     ...
1075  0.254860  0.463591  0.254860  ...  0.328327  1.364823  t-SC-s
1076  0.272198  0.474163  0.251638  ...  0.293435  1.364478  t-SC-s
1077  0.228700  0.395179  0.234118  ...  0.355213  1.430825  t-SC-s
1078  0.221242  0.412894  0.243974  ...  0.365353  1.404031  t-SC-s
1079  0.302626  0.461059  0.256564  ...  0.365278  1.370999  t-SC-s

[612 rows x 77 columns]


In [ ]:
X_datos = Datos_Iniciales.iloc[:,:-1].values
Y_datos = Datos_Iniciales.iloc[:,-1].values
data = X_datos
y_digits = Y_datos

n_samples, n_features = data.shape
n_digits = len(np.unique(y_digits))
labels = y_digits

sample_size = 300

print("n_digits: %d, \t n_samples %d, \t n_features %d"
      % (n_digits, n_samples, n_features))


n_digits: 8, 	 n_samples 612, 	 n_features 76


# Clusters

## Funciones:

### Función **bench_k_means**:

In [ ]:
def bench_k_means(estimator, name, data):
    t0 = time()
    estimator.fit(data)
    print('%-9s\t%.2fs\t%i\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_),
             metrics.silhouette_score(data, estimator.labels_,
                                      metric='euclidean',
                                      sample_size=sample_size), estimator.n_clusters))

### Función **bench_DBSCAN**:

In [ ]:
def bench_DBSCAN(estimator, name, data, uso_outliers):
    t0 = time()
    estimator.fit(data)
    labels_est = estimator.labels_
    labels_originales = labels
    if uso_outliers == False:
      labels_new = np.where(labels_est==-1, None, labels_est)
      # a partir de aqui, experimentar:
      index_non_none = [i for i, val in enumerate(labels_new) if val != None]
      labels_originales_new = np.zeros(len(index_non_none))
      labels_originales_new = [labels_originales[i] for i in index_non_none]
      labels_originales = labels_originales_new
      labels_est = np.zeros(len(index_non_none))
      labels_est = [labels_new[i] for i in index_non_none]
      data_new = data[index_non_none]
      data = data_new
      tamano = len(set(labels_est))
      if tamano == 0 or tamano == 1:
        print('%-9s\t%.2fs\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%.3f'
              % (name, (time() - t0), 'NaN',
                'NaN',
                'NaN',
                'NaN',
                'NaN',
                'NaN',
                'NaN',
                tamano))
      else:
        #print(labels_est)
        print('%-9s\t%.2fs\t%s\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
              % (name, (time() - t0), 'NaN',
                metrics.homogeneity_score(labels_originales, labels_est),
                metrics.completeness_score(labels_originales, labels_est),
                metrics.v_measure_score(labels_originales, labels_est),
                metrics.adjusted_rand_score(labels_originales, labels_est),
                metrics.adjusted_mutual_info_score(labels_originales,  labels_est),
                metrics.silhouette_score(data, labels_est,
                                      metric='euclidean',
                                      sample_size=None), tamano))
    else:
      tamano = len(set(labels_est))
      if tamano == 0 or tamano == 1:
        print('%-9s\t%.2fs\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%.3f'
              % (name, (time() - t0), 'NaN',
                'NaN',
                'NaN',
                'NaN',
                'NaN',
                'NaN',
                'NaN',
                tamano))
      else:
        #print(labels_est)
        print('%-9s\t%.2fs\t%s\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
              % (name, (time() - t0), 'NaN',
                metrics.homogeneity_score(labels_originales, labels_est),
                metrics.completeness_score(labels_originales, labels_est),
                metrics.v_measure_score(labels_originales, labels_est),
                metrics.adjusted_rand_score(labels_originales, labels_est),
                metrics.adjusted_mutual_info_score(labels_originales,  labels_est),
                metrics.silhouette_score(data, labels_est,
                                      metric='euclidean',
                                      sample_size=None), tamano))

In [ ]:
bench_DBSCAN(DBSCAN(eps = 0.1, min_samples = 5), name="dbscan", data=data, uso_outliers=False)

dbscan   	0.03s	NaN	NaN	NaN	NaN	NaN	NaN	NaN	0.000


## Código principal:

## Cálculo de clusters

Hacer pruebas con distintas variantes de algoritmos:
* a. K-Means (con inicialización al azar)
* b. K-means++ 
* c. DBSCAN con épsilon por defecto
* d. DBSCAN con épsilon 0.7
* e. DBSCAN con épsilon 0.2
* f. DBSCAN con épsilon por defecto, agregando outliers a cluster extra
* g. DBSCAN con épsilon 0.7, agregando outliers a cluster extra
* h. DBSCAN con épsilon 0.2, agregando outliers a cluster extra
* i. Clustering aglomerativo
* j. Repetir todas las pruebas anteriores, después de usar PCA sobre los datos para reducirlos a 2 dimensiones.

In [ ]:
print(90 * '_')
print('Sin PCA:')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil.\tclusters')
## SIN PCA
# K-Means random init
bench_k_means(KMeans(init='random', n_clusters=n_digits, n_init=10),
              name="random", data=data)
# K-means ++
bench_k_means(KMeans(init='k-means++', n_clusters=n_digits, n_init=10),
              name="k-means++", data=data)
# DBSCAN, epsilon por defecto
bench_DBSCAN(DBSCAN(min_samples = 5), name="dbscan_F_df", data=data, uso_outliers=False)
# DBSCAN con eps = 0.7
bench_DBSCAN(DBSCAN(eps = 0.7, min_samples = 5), name="dbscan_F_0.7", data=data, uso_outliers=False)
# DBSCAN con eps = 0.2
bench_DBSCAN(DBSCAN(eps = 0.2, min_samples = 5), name="dbscan_F_0.2", data=data, uso_outliers=False)
# DBSCAN, epsilon por defecto, outliers = True
bench_DBSCAN(DBSCAN(min_samples = 5), name="dbscan_T_def", data=data, uso_outliers=True)
# DBSCAN con eps = 0.7, outliers = True
bench_DBSCAN(DBSCAN(eps = 0.7, min_samples = 5), name="dbscan_T_0.7", data=data, uso_outliers=True)
# DBSCAN con eps = 0.2, outliers = True
bench_DBSCAN(DBSCAN(eps = 0.2, min_samples = 5), name="dbscan_T_0.2", data=data, uso_outliers=True)
# Clustering Aglomerativo
bench_DBSCAN(AgglomerativeClustering(n_clusters=n_digits,linkage="ward"), name="Agglo", data=data, uso_outliers=True)
##############################################################################
print('')
## CON PCA
print('Con PCA:')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsil.\tclusters')
pca = PCA(n_components=2).fit(data)
data_pca = pca.transform(data)
# K-Means random init
bench_k_means(KMeans(init='random', n_clusters=n_digits, n_init=10),
              name="random_PCA", data=data_pca)
# K-means ++
bench_k_means(KMeans(init='k-means++', n_clusters=n_digits, n_init=10),
              name="k-means++_PCA", data=data_pca)
# DBSCAN, epsilon por defecto
bench_DBSCAN(DBSCAN(min_samples = 5), name="dbscan_F_df", data=data_pca, uso_outliers=False)
# DBSCAN con eps = 0.7
bench_DBSCAN(DBSCAN(eps = 0.7, min_samples = 5), name="dbscan_F_0.7", data=data_pca, uso_outliers=False)
# DBSCAN con eps = 0.2
bench_DBSCAN(DBSCAN(eps = 0.2, min_samples = 5), name="dbscan_F_0.2", data=data_pca, uso_outliers=False)
# DBSCAN, epsilon por defecto, outliers = True
bench_DBSCAN(DBSCAN(min_samples = 5), name="dbscan_T_df_P", data=data_pca, uso_outliers=True)
# DBSCAN con eps = 0.7, outliers = True
bench_DBSCAN(DBSCAN(eps = 0.7, min_samples = 5), name="DBSCAN_T_0.7", data=data_pca, uso_outliers=True)
# DBSCAN con eps = 0.2, outliers = True
bench_DBSCAN(DBSCAN(eps = 0.2, min_samples = 5), name="DBSCAN_T_0.2", data=data_pca, uso_outliers=True)
# Clustering Aglomerativo
bench_DBSCAN(AgglomerativeClustering(n_clusters=n_digits,linkage="ward"), name="Agglo", data=data_pca, uso_outliers=True)
print(90 * '_')


__________________________________________________________________________________________
Sin PCA:
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil.	clusters
random   	0.10s	1125	0.296	0.298	0.297	0.140	0.283	0.208	8.000
k-means++	0.18s	1122	0.305	0.306	0.305	0.158	0.291	0.222	8.000
dbscan_F_df	0.05s	NaN	1.000	0.705	0.827	0.539	0.771	0.693	12.000
dbscan_F_0.7	0.06s	NaN	0.975	0.536	0.692	0.263	0.630	0.373	43.000
dbscan_F_0.2	0.04s	NaN	NaN	NaN	NaN	NaN	NaN	NaN	0.000
dbscan_T_def	0.05s	NaN	0.107	0.365	0.165	0.002	0.113	-0.349	13.000
dbscan_T_0.7	0.06s	NaN	0.579	0.425	0.490	0.072	0.422	-0.006	44.000
dbscan_T_0.2	0.04s	NaN	NaN	NaN	NaN	NaN	NaN	NaN	1.000
Agglo    	0.02s	NaN	0.352	0.356	0.354	0.168	0.341	0.159	8.000

Con PCA:
init		time	inertia	homo	compl	v-meas	ARI	AMI	sil.	clusters
random_PCA	0.08s	346	0.214	0.217	0.215	0.099	0.199	0.363	8.000
k-means++_PCA	0.11s	345	0.215	0.217	0.216	0.098	0.200	0.345	8.000
dbscan_F_df	0.01s	NaN	NaN	NaN	NaN	NaN	NaN	NaN	1.000
dbscan_F_0.7	0.01s	NaN	NaN	NaN	N